In [ ]:
import pandas as pd

In [ ]:
#읽어오고 전처리 및 숫자화 하기
df = pd.read_csv('노원구_환경데이터.csv', index_col = 0)
df = df.reset_index(drop = True)
df = df.loc[df['pm25']!='pm25',:]
df.pm25 = pd.to_numeric(df.pm25)
df.pm10 = pd.to_numeric(df.pm10)
df.tm = pd.to_numeric(df.tm)
df.flag = pd.to_numeric(df.flag)
df.noise = pd.to_numeric(df.noise)
df.humi = pd.to_numeric(df.humi)
df.temp = pd.to_numeric(df.temp)

In [ ]:
#년/월/일 만 구하기
def date_cuter(dates):
    datelist=[]
    for idx in dates:
        date = str(idx)
        datelist.append(date[:8])
    return datelist

In [ ]:
df['date'] = date_cuter(df.tm)
df['date'] = pd.to_datetime(df['date'])
df.index = df.date
df = df.drop(columns = 'flag')

In [ ]:
#lableing for pm2.5(편의상 25로 되어있음)
df['label_pm25'] = 0
df['label_pm10'] = 0
df_temp0 = df.loc[df.pm25 <= 15,:]
df_temp0['label_pm25'] = 0
df_temp1 = df.loc[df.pm25 >= 16,:]
df_temp1 = df_temp1.loc[df_temp1.pm25 <= 35,:]
df_temp1['label_pm25'] = 1
df_temp2 = df.loc[df.pm25 <= 75,:]
df_temp2 = df_temp2.loc[df_temp2.pm25 >= 36,:]
df_temp2['label_pm25'] = 2
df_temp3 = df.loc[df.pm25 >= 76,:]
df_temp3['label_pm25'] = 3
df_tp=df_temp0.append(df_temp1).append(df_temp2).append(df_temp3)

In [ ]:
#labeling for pm10(미세먼지)
df_temp0 = df_tp.loc[df_tp.pm10 <= 30,:]
df_temp0['label_pm10'] = 0
df_temp1 = df_tp.loc[df_tp.pm10 >= 31,:]
df_temp1 = df_temp1.loc[df_temp1.pm10 <= 80,:]
df_temp1['label_pm10'] = 1
df_temp2 = df_tp.loc[df_tp.pm10 <= 150,:]
df_temp2 = df_temp2.loc[df_temp2.pm10 >= 81,:]
df_temp2['label_pm10'] = 2
df_temp3 = df_tp.loc[df_tp.pm10 >= 151,:]
df_temp3['label_pm10'] = 3
df_labeled=df_temp0.append(df_temp1).append(df_temp2).append(df_temp3)

In [ ]:
#전처리 덜된 값 제거
df_labeled = df_labeled.loc[df_labeled.pm25 != -999,:]

In [ ]:
#Save 및 확인
df_labeled.to_csv("노원구_labeled.csv", mode='w', header=True)
print(df_labeled.loc[df_labeled['serial'] == 'V10O1610102',:].describe())

In [ ]:
#Daily 평균 값으로 데이터 프레임 만들기
df_mean = df_labeled.groupby(df_labeled.index).mean()
df_mean.sort_values('pm10', ascending=False).head(20)

In [ ]:
#그래프화
import matplotlib.pyplot as plt
%matplotlib inline
df_mean.pm10.plot.line()